# 모듈 import

In [ ]:
import os
import IPython
import multiprocessing
import copy
import pickle
import warnings
from datetime import datetime
from time import time
from matplotlib import font_manager as fm, rc, rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import re

import numpy as np
from numpy import array, nan, random as rnd, where as which
import pandas as pd
from pandas import DataFrame as dataframe, Series as series, isna, read_csv
from pandas.tseries.offsets import DateOffset
from scipy.special import boxcox1p
from scipy.stats import skew
from scipy import stats

from sklearn import preprocessing as prep
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split as tts, GridSearchCV as GridTuner, StratifiedKFold, KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn import metrics
from sklearn.pipeline import make_pipeline

from sklearn import linear_model as lm
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as qda
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
from sklearn import neighbors as knn
from sklearn import ensemble

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import metrics as tf_metrics
from tensorflow.keras import callbacks as tf_callbacks
from tqdm.keras import TqdmCallback
from scikeras.wrappers import KerasClassifier, KerasRegressor
import tensorflow_addons as tfa
import keras_tuner as kt
from keras_tuner import HyperModel

# display setting
warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

# font setting
font_path = 'myfonts/NanumSquareB.ttf'
font_obj = fm.FontProperties(fname=font_path, size=12).get_name()
rc('font', family=font_obj)

# %reset -f


# 기본 사용함수

In [ ]:
# ===== utility functions =====

# label encoding for categorical column with excepting na value
class MyLabelEncoder:
    def __init__(self, preset={}):
        # dic_cat format -> {"col_name": {"value": replace}}
        self.dic_cat = preset
    def fit_transform(self, data_x, col_names):
        tmp_x = copy.deepcopy(data_x)
        for i in col_names:
            # type check
            if not ((tmp_x[i].dtype.name == "object") or (tmp_x[i].dtype.name == "category")):
                print(F"WARNING : {i} is not object or category")
            # if key is not in dic, update dic
            if i not in self.dic_cat.keys():
                tmp_dic = dict.fromkeys(sorted(set(tmp_x[i]).difference([nan])))
                label_cnt = 0
                for j in tmp_dic.keys():
                    tmp_dic[j] = label_cnt
                    label_cnt += 1
                self.dic_cat[i] = tmp_dic
            # transform value which is not in dic to nan
            tmp_x[i] = tmp_x[i].astype("object")
            conv = tmp_x[i].replace(self.dic_cat[i])
            for conv_idx, j in enumerate(conv):
                if j not in self.dic_cat[i].values():
                    conv[conv_idx] = nan
            # final return
            tmp_x[i] = conv.astype("float")
        return tmp_x
    def transform(self, data_x, col_names):
        tmp_x = copy.deepcopy(data_x)
        for i in col_names:
            if not ((tmp_x[i].dtype.name == "object") or (tmp_x[i].dtype.name == "category")):
                print(F"WARNING : {i} is not object or category")
            # transform value which is not in dic to nan
            tmp_x[i] = tmp_x[i].astype("object")
            conv = tmp_x[i].replace(self.dic_cat[i])
            for conv_idx, j in enumerate(conv):
                if j not in self.dic_cat[i].values():
                    conv[conv_idx] = nan
            # final return
            tmp_x[i] = conv.astype("float")
        return tmp_x
    def clear(self, dic_cat={}):
        self.dic_cat = dic_cat

class MyOneHotEncoder:
    def __init__(self, label_preset={}):
        self.dic_cat = {}
        self.label_preset = label_preset
    def fit_transform(self, data_x, col_names):
        tmp_x = dataframe()
        for i in data_x:
            if i not in col_names:
                tmp_x = pd.concat([tmp_x, dataframe(data_x[i])], axis=1)
            else:
                if not ((data_x[i].dtype.name == "object") or (data_x[i].dtype.name == "category")):
                    print(F"WARNING : {i} is not object or category")
                self.dic_cat[i] = OneHotEncoder(sparse=False, handle_unknown="ignore")
                conv = self.dic_cat[i].fit_transform(dataframe(data_x[i])).astype("int")
                col_list = []
                for j in self.dic_cat[i].categories_[0]:
                    if i in self.label_preset.keys():
                        for k, v in self.label_preset[i].items():
                            if v == j:
                                col_list.append(str(i) + "_" + str(k))
                    else:
                        col_list.append(str(i) + "_" + str(j))
                conv = dataframe(conv, columns=col_list)
                tmp_x = pd.concat([tmp_x, conv], axis=1)
        return tmp_x
    def transform(self, data_x, col_names):
        tmp_x = dataframe()
        for i in data_x:
            if not i in col_names:
                tmp_x = pd.concat([tmp_x, dataframe(data_x[i])], axis=1)
            else:
                if not ((data_x[i].dtype.name == "object") or (data_x[i].dtype.name == "category")):
                    print(F"WARNING : {i} is not object or category")
                conv = self.dic_cat[i].transform(dataframe(data_x[i])).astype("int")
                col_list = []
                for j in self.dic_cat[i].categories_[0]:
                    if i in self.label_preset.keys():
                        for k, v in self.label_preset[i].items():
                            if v == j: col_list.append(str(i) + "_" + str(k))
                    else:
                        col_list.append(str(i) + "_" + str(j))
                conv = dataframe(conv, columns=col_list)
                tmp_x = pd.concat([tmp_x, conv], axis=1)
        return tmp_x
    def clear(self, dic_cat={}, label_preset={}):
        self.dic_cat = dic_cat
        self.label_preset = label_preset

class MyKNNImputer:
    def __init__(self, k=5):
        self.imputer = KNNImputer(n_neighbors=k)
        self.cat_dic = {}
        self.naidx_dix = {}
    def fit_transform(self, x, y, cat_vars=None):
        for i in cat_vars:
            self.cat_dic[i] = diff(list(sorted(set(x[i]))), [nan])
            self.naidx_dix[i] = list(which(array(x[i].isna()))[0])
        x_imp = dataframe(self.imputer.fit_transform(x, y), columns=x.columns)

        # if imputed categorical value are not in the range, adjust the value
        for i in cat_vars:
            x_imp[i] = x_imp[i].apply(lambda x: int(round(x, 0)))
            for j in self.naidx_dix[i]:
                if x_imp[i][j] not in self.cat_dic[i]:
                    if x_imp[i][j] < self.cat_dic[i][0]:
                        x_imp[i][self.naidx_dix[i]] = self.cat_dic[i][0]
                    elif x_imp[i][j] > self.cat_dic[i][0]:
                        x_imp[i][self.naidx_dix[i]] = self.cat_dic[i][len(self.cat_dic[i]) - 1]
        return x_imp
    def transform(self, x):
        for i in self.cat_dic.keys():
            self.naidx_dix[i] = list(which(array(x[i].isna()))[0])
        x_imp = dataframe(self.imputer.transform(x), columns=x.columns)

        # if imputed categorical value are not in the range, adjust the value
        for i in self.cat_dic.keys():
            x_imp[i] = x_imp[i].apply(lambda x: int(round(x, 0)))
            for j in self.naidx_dix[i]:
                if x_imp[i][j] not in self.cat_dic[i]:
                    if x_imp[i][j] < self.cat_dic[i][0]:
                        x_imp[i][self.naidx_dix[i]] = self.cat_dic[i][0]
                    elif x_imp[i][j] > self.cat_dic[i][0]:
                        x_imp[i][self.naidx_dix[i]] = self.cat_dic[i][len(self.cat_dic[i]) - 1]
        return x_imp
    def clear(self, cat_dic={}, naidx_dix={}):
        self.cat_dic = cat_dic
        self.naidx_dix = naidx_dix

def easyIO(x=None, path=None, op="r"):
    tmp = None
    if op == "r":
        with open(path, "rb") as f:
            tmp = pickle.load(f)
        return tmp
    elif op == "w":
        tmp = {}
        print(x)
        if type(x) is dict:
            for k in x.keys():
                if "MLP" in k:
                    tmp[k] = {}
                    for model_comps in x[k].keys():
                        if model_comps != "model":
                            tmp[k][model_comps] = copy.deepcopy(x[k][model_comps])
                    print(F"INFO : {k} model is removed (keras)")
                else:
                    tmp[k] = x[k]
        if input("Write [y / n]: ") == "y":
            with open(path, "wb") as f:
                pickle.dump(tmp, f)
            print("operation success")
        else:
            print("operation fail")
    else:
        print("Unknown operation type")

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]

def orderElems(for_order, using_ref):
    return [i for i in using_ref if i in for_order]
# concatenate by row

def ccb(df1, df2):
    if type(df1) == series:
        tmp_concat = series(pd.concat([dataframe(df1), dataframe(df2)], axis=0, ignore_index=True).iloc[:,0])
        tmp_concat.reset_index(drop=True, inplace=True)
    elif type(df1) == dataframe:
        tmp_concat = pd.concat([df1, df2], axis=0, ignore_index=True)
        tmp_concat.reset_index(drop=True, inplace=True)
    elif type(df1) == np.ndarray:
        tmp_concat = np.concatenate([df1, df2], axis=0)
    else:
        print("Unknown Type: return 1st argument")
        tmp_concat = df1
    return tmp_concat

def change_width(ax, new_value):
    for patch in ax.patches :
        current_width = patch.get_width()
        adj_value = current_width - new_value
        # we change the bar width
        patch.set_width(new_value)
        # we recenter the bar
        patch.set_x(patch.get_x() + adj_value * .5)
        
def dispPerformance(result_dic, result_metrics):
    perf_table = dataframe(columns=result_metrics)
    for k, v in result_dic.items():
        perf_table = pd.concat([perf_table, v["performance"]], ignore_index=True, axis=0)
    print(perf_table)
    return perf_table


# from sklearn import datasets

# 1. dart_fss api로 rawdata 불러오기

In [ ]:
import dart_fss as dart
from pykrx import stock
# # dart_fss_classifier Plugin 불러오기 (IFRS 도입(2011년) 전의 재무제표와의 호환성)
# import dart_fss_classifier
# # Attach plugin
# assert dart_fss_classifier.attached_plugin() == True
api_key = '730e8899e231f24386cdbff47d900047cf016caf'
dart.set_api_key(api_key)

#전체 기업재무정보 리스트
corp_list = dart.corp.CorpList()

# 2. 데이터셋 기본설정 함수

*   원본 재무제표 로드 함수
*   재무제표 columns 정리(수, 일자 매칭) 함수
*   stock code 로드 함수




In [ ]:
#1-- raw data 로드 함수
def getFinancialStatement(tickers, from_date="20140101", end_date="20210930", report_type=["annual"]):
    # create dictionary for dataset
    # 기업들을 저장할 딕셔너리 초기화
    fs_dic = dict.fromkeys(tickers, None)

    # for loop on tickers
    for i in tickers:
        print("Financial Statement Loading --->", i)
        try:
            # 각 기업의 재무데이터를 저장할 딕셔너리 초기화
            fs_dic[i] = {}
            # 재무제표 로딩
            corp_obj = corp_list.find_by_corp_name(i, exactly=True)[0]. \
                extract_fs(bgn_de=from_date, end_de=end_date, report_tp=report_type, separator=False)
            fs_dic[i]["bs"] = corp_obj["bs"]
            fs_dic[i]["is"] = corp_obj["is"]
            fs_dic[i]["cis"] = corp_obj["cis"]
            fs_dic[i]["cf"] = corp_obj["cf"]
            fs_dic[i]["metrics"] = {}
            fs_dic[i]["stock_code"] = {}
        except:
            # 만약 로딩 시 에러가 발생하면 해당 키 (해당 기업)을 딕셔너리에서 삭제
            print('EXCEPTION : Error occurs, delete the key')
            del fs_dic[i]
    return fs_dic

#2-- 재무제표 데이터의 적절한 컬럼을 추출하는 함수 (2차원 컬럼 -> 1차원 컬럼)
def clearFinancialStatement(fs):
    fs_copy = copy.deepcopy(fs)
    try:
        for i in list(fs.keys()):
            print("===== Columns clearing", i, "=====")
            # label_ko 열에서 한글만 추출하기 위한 re 조건식 지정
            expression_hangul = re.compile('[^가-힣+]')

            # Balance Sheet
            # 2차원 컬럼에서 필요한 열을 추출해 저장할 리스트 생성
            selected_idx = []
            selected_column = []
            # 2중 컬럼에 대한 for loop 수행
            for idx, name in enumerate(fs_copy[i]["bs"]):
                # 튜플 형식으로 넘어오는 name 에 대해 검사 수행
                # 만약 label_ko (매출액, 당기순이익 등의 계정과목이 저장된 열) 이면 저장
                if name[1] == "label_ko":
                    selected_idx.append(idx)
                    selected_column.append(name[1])
                # 일자의 경우 튜플 형식으로 넘어오므로, 튜플 확인 조건문 수행
                # 만약 튜플이고, 해당 튜플의 첫번쨰 튜플 원소가 '연결재무제표' 인 경우 저장 (오타단어 포함)
                if type(name[1]) == tuple and (name[1][0] == "연결재무제표" or name[1][0] == "연결재무재표"):
                    selected_idx.append(idx)
                    selected_column.append(name[0])
            # 반복문의 통해 선택된 열 인덱스 및 열 이름으로 새로운 데이터프레임 생성
            tmp_df = fs_copy[i]["bs"].iloc[:, selected_idx]
            tmp_df.columns = selected_column
            # label_ko 의 문자열에 대해 한글만 추출한 뒤 공백을 모두 제거
            tmp_df["label_ko"] = tmp_df["label_ko"].apply(lambda x: expression_hangul.sub("", x).replace(" ", ""))
            # na 및 공백을 모두 0으로 대치
            fs_copy[i]["bs"] = tmp_df.fillna(0.0)
            fs_copy[i]["bs"].replace(["", " "], 0.0, inplace=True)
            # 각 컬럼의 일자를 YYYYMMDD 형식으로 통일 (fiscal year 의 마지막 일자)
            tmp_cols = ["label_ko"]
            for j in fs_copy[i]["bs"].columns[1:]:
                tmp_cols.append(j.split("-")[-1])
            fs_copy[i]["bs"].columns = tmp_cols
            # 중복 컬럼이 있는 데이터에 대해서는 0이 제일 적은 열만 선택
            dup_cols = fs_copy[i]["bs"].columns.value_counts().index[fs_copy[i]["bs"].columns.value_counts() > 1]
            if len(dup_cols) != 0:
                print("WARNING : duplicated date columns exist (column which has the least zero values is selected)")
                min_zero_col = np.inf
                drop_cols = []
                for j in dup_cols:
                    dup_col_idx = which(fs_copy[i]["bs"].columns == j)[0]
                    for z in dup_col_idx:
                        if (fs_copy[i]["bs"].iloc[:, z] == 0).sum() <= min_zero_col:
                            min_zero_col = z
                    drop_cols = drop_cols + diff(dup_col_idx, [z])
                fs_copy[i]["bs"] = fs_copy[i]["bs"].iloc[:, diff(list(range(len(fs_copy[i]["bs"].columns))), drop_cols)]


            # 손익계산서가 None 이면 포괄손익계산서를 사용
            # 손익계산서가 None 이 아니면 포괄손익계산서(cis) 키값에 대입 후 손익계산서(is) 키는 사용 하지 않음
            if fs_copy[i]["is"] is None:
                # Consolidated Income Statement
                selected_idx_cis = []
                selected_column_cis = []
                for idx, name in enumerate(fs_copy[i]["cis"]):
                    if name[1] == "label_ko":
                        selected_idx_cis.append(idx)
                        selected_column_cis.append(name[1])
                    if type(name[1]) == tuple and (name[1][0] == "연결재무제표" or name[1][0] == "연결재무재표"):
                        selected_idx_cis.append(idx)
                        selected_column_cis.append(name[0])
                tmp_df = fs_copy[i]["cis"].iloc[:, selected_idx_cis]
                tmp_df.columns = selected_column_cis
                tmp_df["label_ko"] = tmp_df["label_ko"].apply(lambda x: expression_hangul.sub("", x).replace(" ", ""))
                fs_copy[i]["cis"] = tmp_df.fillna(0.0)
                fs_copy[i]["cis"].replace(["", " "], 0.0, inplace=True)
            else:
                # Income Statement
                selected_idx_cis = []
                selected_column_cis = []
                for idx, name in enumerate(fs_copy[i]["is"]):
                    if name[1] == "label_ko":
                        selected_idx_cis.append(idx)
                        selected_column_cis.append(name[1])
                    if type(name[1]) == tuple and (name[1][0] == "연결재무제표" or name[1][0] == "연결재무재표"):
                        selected_idx_cis.append(idx)
                        selected_column_cis.append(name[0])
                tmp_df = fs_copy[i]["is"].iloc[:, selected_idx_cis]
                tmp_df.columns = selected_column_cis
                tmp_df["label_ko"] = tmp_df["label_ko"].apply(lambda x: expression_hangul.sub("", x).replace(" ", ""))
                fs_copy[i]["cis"] = tmp_df.fillna(0.0)
                fs_copy[i]["cis"].replace(["", " "], 0.0, inplace=True)
            del fs_copy[i]["is"]

            tmp_cols = ["label_ko"]
            for j in fs_copy[i]["cis"].columns[1:]:
                tmp_cols.append(j.split("-")[-1])
            fs_copy[i]["cis"].columns = tmp_cols

            dup_cols = fs_copy[i]["cis"].columns.value_counts().index[fs_copy[i]["cis"].columns.value_counts() > 1]
            if len(dup_cols) != 0:
                print("WARNING : duplicated date columns exist (column which has the least zero values is selected)")
                min_zero_col = np.inf
                drop_cols = []
                for j in dup_cols:
                    dup_col_idx = which(fs_copy[i]["cis"].columns == j)[0]
                    for z in dup_col_idx:
                        if (fs_copy[i]["cis"].iloc[:, z] == 0).sum() <= min_zero_col:
                            min_zero_col = z
                    drop_cols = drop_cols + diff(dup_col_idx, [z])
                fs_copy[i]["cis"] = fs_copy[i]["cis"].iloc[:, diff(list(range(len(fs_copy[i]["cis"].columns))), drop_cols)]

            # Cash Flow Chart
            selected_idx_cf = []
            selected_column_cf = []
            for idx, name in enumerate(fs_copy[i]["cf"]):
                if name[1] == "label_ko":
                    selected_idx_cf.append(idx)
                    selected_column_cf.append(name[1])
                if type(name[1]) == tuple and (name[1][0] == "연결재무제표" or name[1][0] == "연결재무재표"):
                    selected_idx_cf.append(idx)
                    selected_column_cf.append(name[0])
            tmp_df = fs_copy[i]["cf"].iloc[:, selected_idx_cf]
            tmp_df.columns = selected_column_cf
            tmp_df["label_ko"] = tmp_df["label_ko"].apply(lambda x: expression_hangul.sub("", x).replace(" ", ""))
            fs_copy[i]["cf"] = tmp_df.fillna(0.0)
            fs_copy[i]["cf"].replace(["", " "], 0.0, inplace=True)

            tmp_cols = ["label_ko"]
            for j in fs_copy[i]["cf"].columns[1:]:
                tmp_cols.append(j.split("-")[-1])
            fs_copy[i]["cf"].columns = tmp_cols

            dup_cols = fs_copy[i]["cf"].columns.value_counts().index[fs_copy[i]["cf"].columns.value_counts() > 1]
            if len(dup_cols) != 0:
                print("WARNING : duplicated date columns exist (column which has the least zero values is selected)")
                min_zero_col = np.inf
                drop_cols = []
                for j in dup_cols:
                    dup_col_idx = which(fs_copy[i]["cf"].columns == j)[0]
                    for z in dup_col_idx:
                        if (fs_copy[i]["cf"].iloc[:, z] == 0).sum() <= min_zero_col:
                            min_zero_col = z
                    drop_cols = drop_cols + diff(dup_col_idx, [z])
                fs_copy[i]["cf"] = fs_copy[i]["cf"].iloc[:, diff(list(range(len(fs_copy[i]["cf"].columns))), drop_cols)]

        return fs_copy
    except:
        print("ERROR : return original")
        return fs


#3-- 각 기업 재무제표의 컬럼수 조정 및 일자 매칭 여부 점검 함수
def alignFinancialStatement(fs, cut_off_year=2019, min_recent_year=3, column_unmatched_exception="remove"):
    '''
    :param fs: 특정 기업의 재무데이터가 있는 딕셔너리
    :param cut_off_year: 재무제표에서 제외시킬 최소년도 (ex. 2019 으로 설정된 경우 2019년 이상인 년도는 모두 drop)
    :param min_recent_year: 최소로 존재해야 할 재무제표 (ex. 2019, 2018 년만 존재할 경우 해당 기업 drop)
    :param column_unmatched_exception: bs, cis, cf 의 컬럼 일자가 맞지 않을 경우 처리 방법을 지정 (default : "remove")
    :return: 정상수행 시 전처리된 컬럼을 가진 fs를 리턴, 오류발생 시 원본 fs 를 리턴
    '''
    fs_copy = copy.deepcopy(fs)
    try:
        for k in list(fs.keys()):
            print("===== Columns preprocessing", k, "=====")
            # balance sheet, income statement, cash flow chart 의 컬럼 수를 비교
            col_length_check = array([fs_copy[k]["bs"].shape[1], fs_copy[k]["cis"].shape[1], fs_copy[k]["cf"].shape[1]])
            column_cutoff = col_length_check.min()
            # 만약 컬럼 수가 다르면 수행
            if col_length_check.var() != 0.0:
                print("WARNING : Column's lengths are not same", col_length_check)
                # 각 재무제표의 첫 열 중 가장 작은 값 및 마지막 열 중 가장 큰 값을 계산 (min_col_first ~ max_col_last 범위의 열 선택이 목적)
                min_col_first = str(min(array([fs_copy[k]["bs"].columns[1], fs_copy[k]["cis"].columns[1], fs_copy[k]["cf"].columns[1]], dtype="int")))
                max_col_last = str(max(array([fs_copy[k]["bs"].columns[-1], fs_copy[k]["cis"].columns[-1], fs_copy[k]["cf"].columns[-1]], dtype="int")))
                # 강제로 가장 적은 열을 가진 재무데이터에 다른 데이터들을 맞추도록 하는 flag 설정
                min_cutoff_flag = False
                # slicing 할 열의 인덱스 위치를 저장할 딕셔너리 생성
                cutoff_index = {"bs": None, "cis": None, "cf": None}
                for i in ["bs", "cis", "cf"]:
                    # 위에서 선정한 min_col_first 및 max_col_last 가 각각의 재무제표 열 안에 존재하는지 확인
                    # 만약 존재 하지 않으면 강제로 가장 작은 열을 가진 재무데이터에 다른 재무데이터들을 마지막 열부터 드랍하여 맞춤
                    if (min_col_first not in fs_copy[k][i].columns) or (max_col_last not in fs_copy[k][i].columns):
                        print("WARNING : an interval date doesn't exist in column sequence")
                        # 강제 설정되었으므로 flag를 True로 변경
                        min_cutoff_flag = True
                        # 모든 재무데이터에 일자가 존재해야하므로 한 경우라도 발생하면 break
                        break
                    else:
                        # 일자가 열에 존재하면 min_col_first 와 max_col_first 의 인덱스 위치를 찾아 cut_off_index 딕셔너리에 저장
                        cutoff_index[i] = (which(fs_copy[k][i].columns == min_col_first)[0][0], which(fs_copy[k][i].columns == max_col_last)[0][0])

                # min_cutoff_flag 가 False 이면
                if min_cutoff_flag:
                    fs_copy[k]["bs"] = fs_copy[k]["bs"].iloc[:, 0:column_cutoff]
                    fs_copy[k]["cis"] = fs_copy[k]["cis"].iloc[:, 0:column_cutoff]
                    fs_copy[k]["cf"] = fs_copy[k]["cf"].iloc[:, 0:column_cutoff]
                else:
                    # 각 재무데이터를 min_col_first ~ max_col_last 일자로 맞춤
                    for i in ["bs", "cis", "cf"]:
                        fs_copy[k][i] = fs_copy[k][i].iloc[:, [0] + list(range(cutoff_index[i][0], cutoff_index[i][1]+1))]
                    length_check_after_cutting = array([fs_copy[k]["bs"].shape[1], fs_copy[k]["cis"].shape[1], fs_copy[k]["cf"].shape[1]])
                    # 만약 중간에 비어있는 일자 떄문에 재무데이터 간 컬럼 갯수가 맞지 않으면 해당 기업 drop (ex. 2019 2018 2016 과 같은 열)
                    if length_check_after_cutting.var() != 0:
                        print("WARNING : lengths are not same after cutting")
                        del fs_copy[k]
                        continue
                    else:
                        column_cutoff = length_check_after_cutting.min()
                # 최종 슬라이싱 된 각 재무데이터의 열을 출력
                print("After cutting :", array([fs_copy[k]["bs"].shape[1], fs_copy[k]["cis"].shape[1], fs_copy[k]["cf"].shape[1]]))

            # bs, cis, cf 의 컬럼의 날짜를 align 하는 부분
            # 열 일자 매칭 검사에 대한 해당 기업 drop 여부 flag
            col_notmatch_flag = False
            # label_ko을 제외한 (계정과목 관련) 데이터에 대해 for loop 수행
            for i in range(1, column_cutoff):
                cols_date = array([fs_copy[k]["bs"].columns[i], fs_copy[k]["cis"].columns[i], fs_copy[k]["cf"].columns[i]]).astype("int32")
                # 만약 각 재무제표들의 컬럼 일자가 맞지 않으면 경고 메시지 출력
                if cols_date.var() != 0:
                    print("WARNING : Cross check error on each columns' date")
                    print(cols_date)
                    # column_unmatched_exception 파라미터가 "remove" 이면 해당 기업 drop
                    if column_unmatched_exception == "remove":
                        print("DELETE : column_unmatched_exception == 'remove'")
                        del fs_copy[k]
                        col_notmatch_flag = True
                    # 한 열이라도 맞지 않으면 작업을 수행할 수 없으므로 한 번 조건 통과 시 break
                    break

            # col_notmath_flag 가 False 이면 수행 (각 재무데이터의 컬럼이 align 되어있으면)
            if not col_notmatch_flag:
                # cut_off_year 미만인 컬럼의 인덱스를 구함 (label_ko 는 강제 포함)
                seleted_cols = list(which(pd.to_datetime(series(fs_copy[k]["bs"].drop("label_ko", axis=1).columns)).dt.year < cut_off_year)[0] + 1)
                # 각 재무제표들을 선택된 인덱스를 슬라이싱 하여 재저장 후 이름을 bs의 열 이름으로 통합
                fs_copy[k]["bs"] = fs_copy[k]["bs"].iloc[:, ([0] + seleted_cols)]
                fs_copy[k]["cis"] = fs_copy[k]["cis"].iloc[:, ([0] + seleted_cols)]
                fs_copy[k]["cf"] = fs_copy[k]["cf"].iloc[:, ([0] + seleted_cols)]
                # 최소 요구년도보다 적은 열만 있는 기업 드랍
                if (fs_copy[k]["bs"].shape[1] < min_recent_year + 1):
                    print("DELETE : financial data is less than", min_recent_year)
                    del fs_copy[k]
                    continue
                # 최소 요구년도 이상의 열 중 첫 열의 년도가 cut_off_year - 1 이 아니면 해당 기업 드랍
                if (pd.to_datetime(fs_copy[k]["bs"].columns[1]).year != cut_off_year-1):
                    print("DELETE : first column is not", cut_off_year-1)
                    del fs_copy[k]
                    continue
                # 계정과목에 해당 하는 값이 모두 0인 경우 (label_ko 열을 제외한 모든 열의 값이 0인 경우) 해당 계정과목을 drop 합니다
                fs_copy[k]["bs"] = fs_copy[k]["bs"][(fs_copy[k]["bs"].iloc[:, 1:] != 0).any(True)]
                fs_copy[k]["bs"].reset_index(drop=True, inplace=True)
                fs_copy[k]["cis"] = fs_copy[k]["cis"][(fs_copy[k]["cis"].iloc[:, 1:] != 0).any(True)]
                fs_copy[k]["cis"].reset_index(drop=True, inplace=True)
                fs_copy[k]["cf"] = fs_copy[k]["cf"][(fs_copy[k]["cf"].iloc[:, 1:] != 0).any(True)]
                fs_copy[k]["cf"].reset_index(drop=True, inplace=True)
        return fs_copy
    except:
        print("ERROR : return original")
        return fs


#4-- 해당 기업의 종목코드를 불러오는 함수
def getStockcode(fs):
    fs_copy = copy.deepcopy(fs)
    for i in list(fs.keys()):
        fs_copy[i]["stock_code"] = corp_list.find_by_corp_name(i, exactly=True)[0].info["stock_code"]
    return fs_copy




# 3. stock sector 별 재무데이터 dictionary에 저장

In [ ]:

sector_list = ["comm_services", "discretionary", "energy",
               "health_care", "industrials", "it", "materials", "staples"]
sector = {}
start_time = time()

#상단의 함수를 이용하여 sector별 데이터 dictionary에 저장
for i in sector_list:
    sector[i] = easyIO(None, "./kdigital_project3/fs_rawdata_" + i + ".pickle", op="r")
    sector[i] = clearFinancialStatement(sector[i])
    sector[i] = alignFinancialStatement(sector[i], cut_off_year=2020, min_recent_year=3, column_unmatched_exception="remove")
    sector[i] = getStockcode(sector[i])

time() - start_time


# 4. 학습데이터에 필요한 features 생성

(1) feature 에 필요한 서브지표 계산 함수

In [ ]:
# 연도별 매출액
def getRevenue(fs_meta, years=3, multidriverException="sum"):
    # 서비스업은 주로 매출액이 아닌 영업수익 계정과목으로 매출실적을 나타냄
    condition_list = ["영업수익", "영업수익매출액"]
    primary_check_list = fs_meta["cis"]["label_ko"].apply(lambda x: (x in condition_list) and ("매출원가" not in x))

    # 영업수익 또는 영업수익매출액 키워드를 찾고, 없으면 제조업과 관련된 계정과목으로 매출실적을 찾음
    if primary_check_list.sum() == 0:
        condition_list = ["매출", "매출액", "수익매출액", '수익']
        primary_check_list = fs_meta["cis"]["label_ko"].apply(lambda x: (x in condition_list or "매출액" in x or "수익매출액" in x) and ("매출원가" not in x))

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["cis"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["cis"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return nan")
            return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cis"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return nan")
        return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cis"].iloc[0, 1:(1 + years)].index)

# 연도별 영업이익
def getOperatingIncome(fs_meta, years=3, multidriverException="first"):
    condition_list = ["영업이익", "영업이익손실", "영업손익", "영업손실"]
    primary_check_list = fs_meta["cis"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["cis"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["cis"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return nan")
            return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                          fs_meta["cis"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return nan")
        return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                      fs_meta["cis"].iloc[0, 1:(1 + years)].index)
        
# 연도별 당기순이익
def getEarnings(fs_meta, years=3, multidriverException="first"):
    condition_list = ["당기순이익", "당기순이익손실", "당기순손실", "당기순손익", "연결당기순손익", "연결당기순이익손실"]
    primary_check_list = fs_meta["cis"]["label_ko"].apply(lambda x: x in condition_list)
    if primary_check_list.sum() == 0:
        condition_list = ["지배기업의소유주에게귀속되는당기순이익손실", "지배기업의소유주지분"]
        primary_check_list = fs_meta["cis"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["cis"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["cis"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return nan")
            return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                          fs_meta["cis"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return nan")
        return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                      fs_meta["cis"].iloc[0, 1:(1 + years)].index)
        
# 연도별 EPS
def getEPS(fs_meta, years=3, multidriverException="first", epsType="either", mixed_eps_fisrt=True):
    '''
    :param fs_meta:
    :param years:
    :param multidriverException:
    :param epsType:
    :param mixed_eps_fisrt:
    :return:
    additional : eps 벡터 중 0 값이 있을경우, 자동적으로 계속 및 중단 eps 의 합으로 대체하는 연산을 수행합니다
    '''

    mixed_eps = ["기본및희석주당보통주순이익손실", "기본및희석주당이익", "기본및희석주당순이익",
                 "기본및희석주당이익손실", "기본및희석주당순손실", "기본및희석주당순이익손실"]
    mixed_continuing_eps = ["계속영업기본및희석주당이익손실", "계속영업기본및희석주당순이익"]
    mixed_discontinued_eps = ["중단영업기본및희석주당이익손실", "중단영업기본및희석주당순이익"]

    basic_eps = ["기본주당이익", "기본주당순이익", "기본주당이익손실", "기본주당순이익손실", "기본주당손익",
                 "기본보통주당순이익", "기본보통주당순이익", "보통주기본주당이익", "기본주당손실", "보통주기본주당순이익"]
    basic_continuing_eps = ["계속영업기본주당이익손실", "계속영업기본주당순손익", "계속영업기본주당손실"]
    basic_discontinued_eps = ["중단영업기본주당이익손실", "중단영업기본주당순손익", "중단영업기본주당손실"]

    diluted_eps = ["희석주당이익", "희석주당순이익", "희석주당이익손실", "희석주당순이익손실", "희석주당손익",
                   "희석보통주당순이익", "희석보통주당순이익", "보통주희석주당이익", "희석주당손실", "보통주희석주당순이익"]
    diluted_continuing_eps = ["계속영업희석주당이익손실", "계속영업희석주당순손익", "계속영업희석주당이익"]
    diluted_discontinued_eps = ["중단영업희석주당이익손실", "중단영업희석주당순손익", "중단영업희석주당손실"]

    # mixed_eps_first == True 이면, 기본 및 희석 eps 를 탐색합니다
    if mixed_eps_fisrt:
        mixed_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in mixed_eps)

        # 기본 및 희석 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
        if mixed_label.sum() > 0:
            # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
            if multidriverException == "first":
                mixed_eps_vector = fs_meta["cis"][mixed_label].iloc[0, 1:(1 + years)]
            # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
            else:
                mixed_eps_vector = fs_meta["cis"][mixed_label].iloc[:, 1:(1 + years)].sum(axis=0)

        else:
            mixed_eps_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                      fs_meta["cis"].iloc[0, 1:(1 + years)].index)

        # 기본 eps vector 에 0인 값이 존재하면,
        # 계속영업 및 중단영업 합으로 대체하는 process를 수행합니다
        if (mixed_eps_vector == 0).sum() > 0:
            continuing_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in mixed_continuing_eps)
            discontinued_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in mixed_discontinued_eps)

            # 계속영업 기본 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
            if continuing_label.sum() > 0:
                continuing_vector = fs_meta["cis"][continuing_label].iloc[0, 1:(1 + years)]
            else:
                continuing_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                           fs_meta["cis"].iloc[0, 1:(1 + years)].index)

            # 중단영업 기본 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
            if discontinued_label.sum() > 0:
                discontinued_vector = fs_meta["cis"][discontinued_label].iloc[0, 1:(1 + years)]
            else:
                discontinued_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                             fs_meta["cis"].iloc[0, 1:(1 + years)].index)

            # 기본 eps vector에 0인 값을 계속영업 eps + 중단영업 eps 값으로 대체합니다
            con_discon_sum = continuing_vector + discontinued_vector
            for idx, value in enumerate(mixed_eps_vector):
                if value == 0:
                    mixed_eps_vector[idx] = con_discon_sum[idx]

        # eps vector 값이 모두 0이면 다음 process를 진행합니다
        if (mixed_eps_vector == 0).all():
            print("Mixed EPS values are all zeros ---> pass")
            pass
        # 아니면 eps vector 를 리턴합니다
        else:
            return mixed_eps_vector
    else:
        pass

    # 기본 eps 탐색
    if epsType == "basic":
        basic_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in basic_eps)

        # 기본 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
        if basic_label.sum() > 0:
            # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
            if multidriverException == "first":
                basic_eps_vector = fs_meta["cis"][basic_label].iloc[0, 1:(1 + years)]
            # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
            else:
                basic_eps_vector = fs_meta["cis"][basic_label].iloc[:, 1:(1 + years)].sum(axis=0)
        else:
            basic_eps_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                      fs_meta["cis"].iloc[0, 1:(1 + years)].index)

        # 기본 eps vector 에 0인 값이 존재하면,
        # 계속영업 및 중단영업 합으로 대체하는 process를 수행합니다
        if (basic_eps_vector == 0).sum() > 0:
            continuing_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in basic_continuing_eps)
            discontinued_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in basic_discontinued_eps)

            # 계속영업 기본 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
            if continuing_label.sum() > 0:
                continuing_vector = fs_meta["cis"][continuing_label].iloc[0, 1:(1 + years)]
            else:
                continuing_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                           fs_meta["cis"].iloc[0, 1:(1 + years)].index)

            # 중단영업 기본 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
            if discontinued_label.sum() > 0:
                discontinued_vector = fs_meta["cis"][discontinued_label].iloc[0, 1:(1 + years)]
            else:
                discontinued_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                             fs_meta["cis"].iloc[0, 1:(1 + years)].index)

            # 기본 eps vector에 0인 값을 계속영업 eps + 중단영업 eps 값으로 대체합니다
            con_discon_sum = continuing_vector + discontinued_vector
            for idx, value in enumerate(basic_eps_vector):
                if value == 0:
                    basic_eps_vector[idx] = con_discon_sum[idx]

        # eps vector 값이 모두 0이면 다음 nan을 리턴합니다
        if (basic_eps_vector == 0).all():
            print("ERROR : EPS values are all zeros ---> return nan")
            return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                          fs_meta["cis"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return basic_eps_vector
    # 희석 eps 탐색
    elif epsType == "diluted":
        diluted_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in diluted_eps)

        # 희석 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
        if diluted_label.sum() > 0:
            # 계정과목이 하나 이상 탐지된 경우 제일 첫번째 열만 사용합니다
            if multidriverException == "first":
                diluted_eps_vector = fs_meta["cis"][diluted_label].iloc[0, 1:(1 + years)]
            # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
            else:
                diluted_eps_vector = fs_meta["cis"][diluted_label].iloc[:, 1:(1 + years)].sum(axis=0)
        else:
            diluted_eps_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                        fs_meta["cis"].iloc[0, 1:(1 + years)].index)

        # 희석 eps vector 에 0인 값이 존재하면,
        # 계속영업 및 중단영업 합으로 대체하는 process를 수행합니다
        if (diluted_eps_vector == 0).sum() > 0:
            continuing_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in diluted_continuing_eps)
            discontinued_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in diluted_discontinued_eps)

            # 계속영업 희석 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
            if continuing_label.sum() > 0:
                continuing_vector = fs_meta["cis"][continuing_label].iloc[0, 1:(1 + years)]
            else:
                continuing_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                           fs_meta["cis"].iloc[0, 1:(1 + years)].index)

            # 중단영업 희석 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
            if discontinued_label.sum() > 0:
                discontinued_vector = fs_meta["cis"][discontinued_label].iloc[0, 1:(1 + years)]
            else:
                discontinued_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                             fs_meta["cis"].iloc[0, 1:(1 + years)].index)

            # 희석 eps vector에 0인 값을 계속영업 eps + 중단영업 eps 값으로 대체합니다
            con_discon_sum = continuing_vector + discontinued_vector
            for idx, value in enumerate(diluted_eps_vector):
                if value == 0:
                    diluted_eps_vector[idx] = con_discon_sum[idx]

        # eps vector 값이 모두 0이면 다음 nan을 리턴합니다
        if (diluted_eps_vector == 0).all():
            print("ERROR : EPS values are all zeros ---> return nan")
            return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                          fs_meta["cis"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return diluted_eps_vector
    # 기본 eps 및 희석 eps 를 둘 다 탐색하여, 기본 eps 가 0 이면 희석 eps 값으로 대체
    else:
        # ===== basic eps 계산 =====
        basic_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in basic_eps)

        # 기본 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
        if basic_label.sum() > 0:
            # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
            if multidriverException == "first":
                basic_eps_vector = fs_meta["cis"][basic_label].iloc[0, 1:(1 + years)]
            # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
            else:
                basic_eps_vector = fs_meta["cis"][basic_label].iloc[:, 1:(1 + years)].sum(axis=0)
        else:
            basic_eps_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                      fs_meta["cis"].iloc[0, 1:(1 + years)].index)

        # 기본 eps vector 에 0인 값이 존재하면,
        # 계속영업 및 중단영업 합으로 대체하는 process를 수행합니다
        if (basic_eps_vector == 0).sum() > 0:
            continuing_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in basic_continuing_eps)
            discontinued_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in basic_discontinued_eps)

            # 계속영업 기본 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
            if continuing_label.sum() > 0:
                continuing_vector = fs_meta["cis"][continuing_label].iloc[0, 1:(1 + years)]
            else:
                continuing_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                           fs_meta["cis"].iloc[0, 1:(1 + years)].index)

            # 중단영업 기본 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
            if discontinued_label.sum() > 0:
                discontinued_vector = fs_meta["cis"][discontinued_label].iloc[0, 1:(1 + years)]
            else:
                discontinued_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                             fs_meta["cis"].iloc[0, 1:(1 + years)].index)

            # 기본 eps vector에 0인 값을 계속영업 eps + 중단영업 eps 값으로 대체합니다
            con_discon_sum = continuing_vector + discontinued_vector
            for idx, value in enumerate(basic_eps_vector):
                if value == 0:
                    basic_eps_vector[idx] = con_discon_sum[idx]

        # ===== diluted eps 대치 =====
        # eps vector 에 0인 값이 하나라도 있으면
        if (basic_eps_vector == 0).sum() > 0:
            print("diluted eps replacement process")
            diluted_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in diluted_eps)

            # 희석 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
            if diluted_label.sum() > 0:
                # 계정과목이 하나 이상 탐지된 경우 제일 첫번째 열만 사용합니다
                if multidriverException == "first":
                    diluted_eps_vector = fs_meta["cis"][diluted_label].iloc[0, 1:(1 + years)]
                # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
                else:
                    diluted_eps_vector = fs_meta["cis"][diluted_label].iloc[:, 1:(1 + years)].sum(axis=0)
            else:
                diluted_eps_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                            fs_meta["cis"].iloc[0, 1:(1 + years)].index)

            # 희석 eps vector 에 0인 값이 존재하면,
            # 계속영업 및 중단영업 합으로 대체하는 process를 수행합니다
            if (diluted_eps_vector == 0).sum() > 0:
                continuing_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in diluted_continuing_eps)
                discontinued_label = fs_meta["cis"]["label_ko"].apply(lambda x: x in diluted_discontinued_eps)

                # 계속영업 희석 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
                if continuing_label.sum() > 0:
                    continuing_vector = fs_meta["cis"][continuing_label].iloc[0, 1:(1 + years)]
                else:
                    continuing_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                               fs_meta["cis"].iloc[0, 1:(1 + years)].index)

                # 중단영업 희석 eps 를 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
                if discontinued_label.sum() > 0:
                    discontinued_vector = fs_meta["cis"][discontinued_label].iloc[0, 1:(1 + years)]
                else:
                    discontinued_vector = series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                                                 fs_meta["cis"].iloc[0, 1:(1 + years)].index)

                # 희석 eps vector에 0인 값을 계속영업 eps + 중단영업 eps 값으로 대체합니다
                con_discon_sum = continuing_vector + discontinued_vector
                for idx, value in enumerate(diluted_eps_vector):
                    if value == 0:
                        diluted_eps_vector[idx] = con_discon_sum[idx]

            # 기본 eps vector에 0인 값을 희석 eps vector의 값으로 대체합니다
            for idx, value in enumerate(basic_eps_vector):
                if value == 0:
                    basic_eps_vector[idx] = diluted_eps_vector[idx]

        # eps vector 값이 모두 0이면 다음 nan을 리턴합니다
        if (basic_eps_vector == 0).all():
            print("ERROR : EPS values are all zeros ---> return nan")
            return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index),
                          fs_meta["cis"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return basic_eps_vector

# 연도별 법인세
def getTax(fs_meta, years=3, multidriverException="first"):
    condition_list = ["법인세의납부", "법인세비용수익", "법인세납부환급", "법인세납부", "법인세비용", "법인세납부액", "법인세의환급지급", "법인세환급",
                      "법인세환급납부", "법인세납부영업", "법인세비용", "법인세납부액환급액"]
    primary_check_list = fs_meta["cf"]["label_ko"].apply(lambda x: x in condition_list or "법인세" in x)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["cf"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["cf"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return 0")
            return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return 0")
        return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)

# 연도별 이자비용
def getInterestExpense(fs_meta, years=3, multidriverException="first"):
    condition_list = ["이자비용", "이자의지급", "이자지급", "이자지급영업"]
    primary_check_list = fs_meta["cf"]["label_ko"].apply(lambda x: x in condition_list or "이자비용" in x or "이자지급" in x)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["cf"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["cf"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)

        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return 0")
            return series([0] * len(fs_meta["cf"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return 0")
        return series([0] * len(fs_meta["cf"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)

# 연도별 감가상각비
def getDepreciationAndAmortization(fs_meta, years=3, multidriverException="sum"):
    condition_list = ["감가상각비", "자산상각비", "대손상각비", "당기순이익조정을위한가감", "조정", "수익비용의조정", "무형자산감가상각비"]
    primary_check_list = fs_meta["cf"]["label_ko"].apply(lambda x: x in condition_list or "자산상각비" in x or "감가상각비" in x)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["cf"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["cf"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return 0")
            return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return 0")
        return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)

# 연도별 자기자본
def getEquity(fs_meta, years=3, multidriverException="first"):
    condition_list = ["자본총계"]
    primary_check_list = fs_meta["bs"]["label_ko"].apply(lambda x: x in condition_list)
    if primary_check_list.sum() == 0:
        condition_list = ["자본"]
        primary_check_list = fs_meta["bs"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["bs"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["bs"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return nan")
            return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return nan")
        return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)

# 연도별 부채
def getDebt(fs_meta, years=3, multidriverException="first"):
    condition_list = ["부채총계"]
    primary_check_list = fs_meta["bs"]["label_ko"].apply(lambda x: x in condition_list)
    if primary_check_list.sum() == 0:
        condition_list = ["부채"]
        primary_check_list = fs_meta["bs"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["bs"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["bs"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return nan")
            return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return nan")
        return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)

# 연도별 영업활동 현금흐름 (조정항목이 계산되지 않은 현금흐름)
def getCFO(fs_meta, years=3, multidriverException="first"):
    condition_list = ["영업활동현금흐름", "영업활동으로인한현금흐름"]
    primary_check_list = fs_meta["cf"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["cf"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["cf"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return nan")
            return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return nan")
        return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)

# 연도별 영업에서 창출된 현금흐름 (조정항목이 미리 계산된 현금흐름)
def getAdjCFO(fs_meta, years=3, multidriverException="first"):
    condition_list = ["영업에서창출된현금흐름", "영업으로부터창출된현금흐름", "영업에서창출된현금", "계속영업에서창출된현금흐름",\
                      "영업활동으로부터창출된현금흐름","영업활동에서창출된현금흐름","영업활동으로창출된현금", "영업으로부터창출된현금" ]

    primary_check_list = fs_meta["cf"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["cf"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["cf"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return 0")
            return series([0] * len(fs_meta["cf"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return 0")
        return series([0] * len(fs_meta["cf"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)

# 연도별 재고자산
def getInventory(fs_meta, years=3, multidriverException="first"):
    condition_list = ['재고자산']
    primary_check_list = fs_meta["bs"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["bs"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["bs"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return 0")
            return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return 0")
        return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)

# 연도별 유동자산
def getCurrentAsset(fs_meta, years=3, multidriverException="first"):
    condition_list = ["유동자산"]
    primary_check_list = fs_meta["bs"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["bs"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["bs"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return nan")
            return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return 0")
        return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)

# 연도별 유동부채
def getCurrentLiab(fs_meta, years=3, multidriverException="first"):
    condition_list = ["유동부채"]
    primary_check_list = fs_meta["bs"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["bs"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["bs"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return nan")
            return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return nan")
        return series([nan] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)

# 연도별 배당금
def getDividend(fs_meta, years=3, multidriverException="first"):
    condition_list = ["배당금지급", "배당금의지급", '배당금의지급등']
    primary_check_list = fs_meta["cf"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["cf"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["cf"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return 0")
            return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return 0")
        return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)

# 연도별 자사주매입금
def getBuyback(fs_meta, years=3, multidriverException="first"):
    condition_list = ["자기주식의처분", '자기주식의취득']
    primary_check_list = fs_meta["cf"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["cf"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["cf"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return 0")
            return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return 0")
        return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["cf"].iloc[0, 1:(1 + years)].index)

# 연도별 매출채권 (not used)
def getBond(fs_meta, years=3, multidriverException="sum"):
    condition_list = ["매출채권및기타채권", "매출채권", '매출채권및기타유동채권', '매출채권및기타채권', '매출채권및대출채권',
                      '매출채권및기타수취채권', '매출채권및수취채권', '매출채권및계약자산', '단기매출채권', '유동매출채권및기타채권']

    primary_check_list = fs_meta["bs"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["bs"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["bs"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return 0")
            return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return 0")
        return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)

# 연도별 현금성자산 (not used)
def getCash(fs_meta, years=3, multidriverException="sum"):
    condition_list = ['현금및현금성자산', '현금']
    primary_check_list = fs_meta["bs"]["label_ko"].apply(lambda x: x in condition_list)

    # 계정과목을 탐색하여 vector를 생성하고, 존재하지 않을 경우 0인 벡터를 생성합니다
    if primary_check_list.sum() > 0:
        if primary_check_list.sum() > 1: print("WARNING : Multi drivers are detected")
        # 계정과목이 하나 이상 탐지된 경우 제일 첫 번째 열을 사용합니다
        if multidriverException == "first":
            output_vector = fs_meta["bs"][primary_check_list].iloc[0, 1:(1 + years)]
        # 계정과목이 여러 개 탐지된 경우 모두 합산한 값을 사용합니다
        else:
            output_vector = fs_meta["bs"][primary_check_list].iloc[:, 1:(1 + years)].sum(axis=0)
        # eps vector 값이 모두 0일 경우 리턴값을 설정합니다
        if (output_vector == 0).all():
            print("INFO : All values are zero ---> return 0")
            return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)
        # 아니면 eps vector 를 리턴합니다
        else:
            return output_vector
    # driver가 찾아지지 않은 경우, 리턴값을 설정합니다
    else:
        print("ERROR : Any driver is not found ---> return 0")
        return series([0] * len(fs_meta["cis"].iloc[0, 1:(1 + years)].index), fs_meta["bs"].iloc[0, 1:(1 + years)].index)
        
# 연도별 일평균 주가 (종가기준)
def getStockPrice(fs_meta, stock_dates=(), item_name="종가", op="mean"):
    if type(stock_dates) in (tuple, list):
        from_date = stock_dates[0]
        to_date = stock_dates[1]
    else:
        from_date = (pd.to_datetime(datetime.now()) - DateOffset(days=stock_dates)).strftime('%Y%m%d')
        to_date = pd.to_datetime(datetime.now()).strftime('%Y%m%d')
    if op == "mean":
        ohlc = stock.get_market_ohlcv_by_date(fromdate=from_date, todate=to_date, ticker=fs_meta["stock_code"]).resample("1Y").mean().transpose()
        ohlc.columns = ohlc.columns.to_series().apply(lambda x: x.strftime("%Y%m%d"))
        ohlc.reset_index(inplace=True)
        ohlc.columns = ["label_ko"] + list(ohlc.columns[1:])
        ohlc.columns.name = None
        ohlc = ohlc[ohlc["label_ko"] == item_name].iloc[0, 1:]
        ohlc.sort_index(ascending=False, inplace=True)
    else:
        ohlc = stock.get_market_ohlcv_by_date(fromdate=from_date, todate=to_date, ticker=fs_meta["stock_code"])
    return ohlc


(2) feature 계산함수

*   성장성 지표
*   안정성 지표
*   수익성 지표
*   가치평가 지표
*   주주환원 지표








In [ ]:

# ===== financial statement =====
# 1. 성장성 지표 (전체기간 평균 증가율)
# 매출액 평균 성장률
# 영입이익 평균 성장률
# EBITDA 평균 성장률
# EPS 평균 성장률
def getRateOfChange(df):
    df = df.iloc[0]
    df.sort_index(inplace=True)
    df = df.replace(0, nan).interpolate()
    return df.pct_change()[1:].mean()
def getGrowthIndicator(fs, years=7):
    fs_copy = copy.deepcopy(fs)
    print(fs_copy.keys())
    for i in fs.keys():
        print(i)
        ##매출액 연평균 성장률
        a = getRevenue(fs_copy[i], years=years)
        a = a.to_frame().T
        # print(growth(a, years))
        try:
            fs_copy[i]["metrics"]["yoy_avg_revenue"] = getRateOfChange(a, len(a.columns))
        except:
            fs_copy[i]["metrics"]["yoy_avg_revenue"] = nan

        ##영업이익 연평균 성장률
        b = getOperatingIncome(fs_copy[i], years=years)
        b = b.to_frame().T
        # print(growth(b, years))
        try:
            fs_copy[i]["metrics"]["yoy_avg_operating_income"] = getRateOfChange(b, len(b.columns))
        except:
            fs_copy[i]["metrics"]["yoy_avg_operating_income"] = nan

        ##EBITDA 연평균 성장률
        if getAdjCFO(fs_copy[i], years).mean() == 0:
            c = getEarnings(fs_copy[i], years) + \
                getDepreciationAndAmortization(fs_copy[i], years) - \
                getInterestExpense(fs_copy[i], years) - \
                getTax(fs_copy[i], years)
        else:
            c = getAdjCFO(fs_copy[i], years) - \
                getInterestExpense(fs_copy[i], years) - \
                getTax(fs_copy[i], years)
        c = c.to_frame().T
        # print(growth(b, years))
        try:
            fs_copy[i]["metrics"]["yoy_avg_ebitda"] = getRateOfChange(c, len(c.columns))
        except:
            fs_copy[i]["metrics"]["yoy_avg_ebitda"] = nan

        ##eps 연평균 성장률
        d = getEPS(fs_copy[i], years)
        d = d.to_frame().T
        # print(growth(c, years))
        try:
            fs_copy[i]["metrics"]["yoy_avg_eps"] = getRateOfChange(d, len(d.columns))
        except:
            fs_copy[i]["metrics"]["yoy_avg_eps"] = nan

    return fs_copy

# 2. 안정성 지표 (최근 3년 평균)
# Quick ratio = (유동자산 - 재고자산) / 유동부채
# 매출채권 회전율 = 매출액 / 매출채권
# 부채비율 = 부채 / 자기자본
# ICR(Interest Coverage Ratio) = EBITDA / 이자비용
def getStabilityIndicator(fs, years=3):
    fs_copy = copy.deepcopy(fs)
    for i in list(fs_copy.keys()):
        print("===== Getting metrics on", i, "=====")
        try:
            fs_copy[i]["metrics"]["quick_ratio"] = ((getCurrentAsset(fs_copy[i], years) - getInventory(fs_copy[i], years)) / \
                                                    getCurrentLiab(fs_copy[i], years)).mean()
        except:
            fs_copy[i]["metrics"]["quick_ratio"] = nan

        try:
            fs_copy[i]["metrics"]["receivable_turnover_ratio"] = (getRevenue(fs_copy[i], years) / getBond(fs_copy[i], years)).mean()
        except:
            fs_copy[i]["metrics"]["receivable_turnover_ratio"] = nan

        try:
            fs_copy[i]["metrics"]["asset_turnover_ratio"] = (getRevenue(fs_copy[i], years) / (getEquity(fs_copy[i], years) + getDebt(fs_copy[i], years))).mean()
        except:
            fs_copy[i]["metrics"]["asset_turnover_ratio"] = nan

        try:
            fs_copy[i]["metrics"]["debt_to_equity_ratio"] = (getDebt(fs_copy[i], years) / getEquity(fs_copy[i], years)).mean()
        except:
            fs_copy[i]["metrics"]["debt_to_equity_ratio"] = nan

        try:
            if getAdjCFO(fs_copy[i], years).mean() == 0:
                ebitda = getEarnings(fs_copy[i], years) + \
                         getDepreciationAndAmortization(fs_copy[i], years) - \
                         getInterestExpense(fs_copy[i], years) - \
                         getTax(fs_copy[i], years)
            else:
                ebitda = getAdjCFO(fs_copy[i], years) - \
                         getInterestExpense(fs_copy[i], years) - \
                         getTax(fs_copy[i], years)
            interest_expense = getInterestExpense(fs_copy[i], years)
            ebitda = ebitda[interest_expense != 0]
            interest_expense = interest_expense[interest_expense != 0]
            fs_copy[i]["metrics"]["icr"] = (ebitda / interest_expense).mean()
        except:
            fs_copy[i]["metrics"]["icr"] = nan
        print("===== Complete =====")
    return fs_copy

# 3. 수익성 지표 (최근 3년 평균)
# 매출액
# 영업이익
# 영업이익 마진
# EBITDA
# EBITDA 마진
# ROE = 당기순이익 / 자기자본
# CFO to Earnings = 영업현금흐름 / 당기순이익
def getProfitabilityIndicator(fs, years=3):
    fs_copy = copy.deepcopy(fs)
    for i in list(fs_copy.keys()):
        print("===== Getting metrics on", i, "=====")
        fs_copy[i]["metrics"]["revenue"] = getRevenue(fs_copy[i], years).mean()
        fs_copy[i]["metrics"]["operating_income"] = getOperatingIncome(fs_copy[i], years).mean()
        fs_copy[i]["metrics"]["operating_margin"] = (getOperatingIncome(fs_copy[i], years) / getRevenue(fs_copy[i], years)).mean()
        if getAdjCFO(fs_copy[i], years).mean() == 0:
            fs_copy[i]["metrics"]["ebitda"] = (getEarnings(fs_copy[i], years) + \
                                               getDepreciationAndAmortization(fs_copy[i], years) - \
                                               getInterestExpense(fs_copy[i], years) - \
                                               getTax(fs_copy[i], years)).mean()
            fs_copy[i]["metrics"]["ebitda_margin"] = ((getEarnings(fs_copy[i], years) + \
                                                       getDepreciationAndAmortization(fs_copy[i], years) - \
                                                       getInterestExpense(fs_copy[i], years) - \
                                                       getTax(fs_copy[i], years)) / getRevenue(fs_copy[i], years)).mean()
        else:
            fs_copy[i]["metrics"]["ebitda"] = (getAdjCFO(fs_copy[i], years) - \
                                               getInterestExpense(fs_copy[i], years) - \
                                               getTax(fs_copy[i], years)).mean()
            fs_copy[i]["metrics"]["ebitda_margin"] = ((getAdjCFO(fs_copy[i], years) - \
                                                       getInterestExpense(fs_copy[i], years) - \
                                                       getTax(fs_copy[i], years)) / getRevenue(fs_copy[i], years)).mean()
        try:
            fs_copy[i]["metrics"]["roe"] = (getEarnings(fs_copy[i], years) / getEquity(fs_copy[i], years)).mean()
        except:
            fs_copy[i]["metrics"]["roe"] = nan

        try:
            fs_copy[i]["metrics"]["cfo_to_earnings"] = (getCFO(fs_copy[i], years) / getEarnings(fs_copy[i], years)).mean()
        except:
            fs_copy[i]["metrics"]["cfo_to_earnings"] = nan
        print("===== Complete =====")
    return fs_copy
    
# 4. 가치평가 지표 (최근 1년)
# PSR = 1주당 주가 / 주당 매출액
# PER = 1주당 주가 / 주당 순이익
def getValuationIndicator(fs, years=1, stock_dates=()):
    fs_copy = copy.deepcopy(fs)
    for i in list(fs_copy.keys()):
        print("===== Getting metrics on", i, "=====")
        try:
            fs_copy[i]["metrics"]["per"] = (getStockPrice(fs_copy[i], stock_dates).values / getEPS(fs_copy[i], years).values).mean()
        except:
            fs_copy[i]["metrics"]["per"] = nan
        print("===== Complete =====")
    return fs_copy
# 5. 주주환원 지표 (최근 3년 평균)
# 주주환원율 = (배당금 + 자사주매입금) / 당기순이익
def getShareholderReturnIndicator(fs, years=3):
    fs_copy = copy.deepcopy(fs)
    for i in list(fs_copy.keys()):
        print("===== Getting metrics on", i, "=====")
        fs_copy[i]["metrics"]["shareholder_return"] = ((getDividend(fs_copy[i], years) + getBuyback(fs_copy[i], years)) / \
                                                       getEarnings(fs_copy[i], years)).mean()
        print("===== Complete =====")
    return fs_copy

def getBeta(portfolio, benchmark):
    return stats.linregress(portfolio.pct_change()[1:].values, benchmark.pct_change()[1:].values)[0]
def getJensenAlpah(fs, benchmark="KOSPI", riskfree_rate=0.02, stock_dates=(), period="q"):
    fs_copy = copy.deepcopy(fs)
    if benchmark == "KOSPI":
        benchmark_df = stock.get_index_ohlcv_by_date(stock_dates[0], stock_dates[1], "1001")["종가"]
        benchmark_df.name = "benchmark"
    else:
        print("Unknown benchmark : return original")
        return fs
    for i in list(fs.keys()):
        try:
            print("===== Get jensen alpha on", i, "=====")
            portfolio_df = getStockPrice(fs_copy[i], stock_dates=(stock_dates[0], stock_dates[1]), op="raw")["종가"]
            portfolio_df.name = "portfolio"

            # 해당 기간의 수익률을 구합니다 (기간 첫 value 와 기간 마지막 value 의 변화율)
            comp_df = pd.concat([benchmark_df, portfolio_df], axis=1, join="inner")
            if period == "m":
                comp_df_resample = comp_df.resample("1M").agg(lambda x: (x[-1] - x[0]) / x[0])
                comp_df_resample["beta"] = comp_df.resample("1M").agg(lambda x: getBeta(x["portfolio"], x["benchmark"]))
            elif period == "q":
                comp_df_resample = comp_df.resample("1Q").agg(lambda x: (x[-1] - x[0]) / x[0])
                comp_df_resample["beta"] = comp_df.resample("1Q").agg(lambda x: getBeta(x["portfolio"], x["benchmark"]))
            elif period == "y":
                comp_df_resample = comp_df.resample("1Y").agg(lambda x: (x[-1] - x[0]) / x[0])
                comp_df_resample["beta"] = comp_df.resample("1Y").agg(lambda x: getBeta(x["portfolio"], x["benchmark"]))
            else:
                comp_df_resample = dataframe([comp_df.agg(lambda x: (x[-1] - x[0]) / x[0])])
                comp_df_resample["beta"] = getBeta(comp_df["portfolio"], comp_df["benchmark"])
                print("Unknown period : 'daily' is set")

            # 기간 평균 젠센알파를 구합니다
            fs_copy[i]["metrics"]["jensen_alpha"] = (comp_df_resample["portfolio"] - (riskfree_rate + comp_df_resample["beta"] * (comp_df_resample["benchmark"] - riskfree_rate))).mean()
            fs_copy[i]["metrics"]["jensen_alpha_class"] = 1 if fs_copy[i]["metrics"]["jensen_alpha"] >= 0 else 0
        except:
            print("ERROR : delete key", i)
            del fs_copy[i]
    return fs_copy
def getSortinoRatio(fs, benchmark="KOSPI", riskfree_rate=0.02, stock_dates=(), period="q"):
    fs_copy = copy.deepcopy(fs)
    if benchmark == "KOSPI":
        benchmark_df = stock.get_index_ohlcv_by_date(stock_dates[0], stock_dates[1], "1001")["종가"]
        benchmark_df.name = "benchmark"
    else:
        print("Unknown benchmark : return original")
        return fs
    for i in list(fs.keys()):
        try:
            print("===== Get sortino ratio on", i, "=====")
            portfolio_df = getStockPrice(fs_copy[i], stock_dates=(stock_dates[0], stock_dates[1]), op="raw")["종가"]
            portfolio_df.name = "portfolio"

            # 해당 기간의 수익률을 구합니다 (기간 첫 value 와 기간 마지막 value 의 변화율)
            comp_df = pd.concat([benchmark_df, portfolio_df], axis=1, join="inner")
            if period == "m":
                comp_df_resample = comp_df.resample("1M").agg(lambda x: (x[-1] - x[0]) / x[0])
                comp_df_resample["downside_std"] = comp_df.pct_change()[1:]["portfolio"].resample("1M").agg(lambda x: x[x < 0].std())
            elif period == "q":
                comp_df_resample = comp_df.resample("1Q").agg(lambda x: (x[-1] - x[0]) / x[0])
                comp_df_resample["downside_std"] = comp_df.pct_change()[1:]["portfolio"].resample("1Q").agg(lambda x: x[x < 0].std())
            elif period == "y":
                comp_df_resample = comp_df.resample("1Y").agg(lambda x: (x[-1] - x[0]) / x[0])
                comp_df_resample["downside_std"] = comp_df.pct_change()[1:]["portfolio"].resample("1Y").agg(lambda x: x[x < 0].std())
            else:
                comp_df_resample = dataframe([comp_df.agg(lambda x: (x[-1] - x[0]) / x[0])])
                comp_df_resample["downside_std"] = comp_df.pct_change()[1:]["portfolio"][comp_df.pct_change()[1:]["portfolio"]<0].std()
                print("Unknown period : 'day' is set")

            # 기간 평균 소티노비율를 구합니다
            fs_copy[i]["metrics"]["sortino_ratio"] = ((comp_df_resample["portfolio"] - riskfree_rate) / comp_df_resample["downside_std"]).mean()
            fs_copy[i]["metrics"]["sortino_ratio_class"] = 1 if fs_copy[i]["metrics"]["sortino_ratio"] >= 0 else 0
        except:
            print("ERROR : delete key", i)
            del fs_copy[i]
    return fs_copy


(3) sector dictionary에 기업별 재무 feature 저장

In [ ]:

for i in sector.keys():
    sector[i] = getGrowthIndicator(sector[i], 7)
    sector[i] = getStabilityIndicator(sector[i])
    sector[i] = getProfitabilityIndicator(sector[i])
    sector[i] = getValuationIndicator(sector[i], 1, ("20190101", "20191231"))
    sector[i] = getShareholderReturnIndicator(sector[i])
    sector[i] = getJensenAlpah(sector[i], riskfree_rate=0.0169, stock_dates=("20200101", "20201231"))
    # sector[i] = getSortinoRatio(sector[i], riskfree_rate=0.0189, stock_dates=("20190101", "20191231"))


(4) sector dict 내의 데이터를 dataframe으로 변환

In [ ]:

def transform_frame(sector_dict, sector_key, col_name):
    df = dataframe(columns=col_name)
    for k, v in sector_dict.items():
        df = pd.concat([df, dataframe([v["metrics"].values()], columns=v["metrics"].keys())], axis=0)
    df["sector"] = sector_key
    df.index = sector_dict.keys()
    df.index.name = "기업명"
    return df
    
# metric 키값 구하기 및 sector 컬럼 결합
# sector iteration
for i in sector.keys():
    # corporate iteration
    for j in sector[i].keys():
        col_name = list(sector[i][j]["metrics"].keys()) + ["sector"]
        break

# 빈데이터 프레임 생성
full_x = dataframe(columns=col_name)
# sector에 있는 데이터 모두 데이터프레임화
for key, item in sector.items():
    a = transform_frame(item, key, col_name)
    full_x = pd.concat([full_x, a], axis=0)
del sector


(5) 생성한 학습데이터셋에 대한 중복 확인 및 NA값 대치

In [ ]:

# 무한대값은 0으로 나누는 연산이 있는 series 에서 발생한 것이므로 na 처리
full_x.replace([np.inf, -np.inf], nan, inplace=True)

dup_table = pd.read_excel("kdigital_project3/중복기업리스트_섹터할당.xlsx")
dup_table.columns = ["name", "sector", "사업부문", "추천섹터"]
dup_table = dup_table.drop(which(dup_table["추천섹터"].isna())[0], axis=0)

# 산업군에 중복으로 들어간 기업들에 대해 drop 수행
drop_list = []
for i in range(full_x.shape[0]):
    if full_x.index[i] in dup_table["name"].values:
        if full_x["sector"][i] != dup_table[full_x.index[i] == dup_table["name"].values]["sector"].values[0]:
            drop_list.append(i)

retain_list = diff(list(range(full_x.shape[0])), drop_list)
full_x = full_x.iloc[retain_list]
corp_index = full_x.index
full_x.reset_index(drop=True, inplace=True)

full_y = full_x["jensen_alpha_class"]
jensen_alpha = full_x["jensen_alpha"]
full_x.drop(["jensen_alpha", "jensen_alpha_class"], axis=1, inplace=True)

full_y.isna().sum()
full_x.isna().sum()
cat_vars = ["sector"]

# sector 컬럼에 대해 label encoding
label_encoder = MyLabelEncoder()
full_x = label_encoder.fit_transform(full_x, cat_vars)

# knn imputer 를 활용한 na값 imputing 
knn_imputer = MyKNNImputer()
full_x = knn_imputer.fit_transform(full_x, full_y, cat_vars)

print(full_x.isna().sum())


# 질문 목록




1. 저희 팀은 특정 시점의 재무데이터를 가지고 미래의 분기평균 젠센알파를 이진분류하려고 합니다. (target은 젠센알파가 0 보다 크거나 같으면 1 아니면 0 변환) 다음과 같은 train 및 test 기간이 적절한지요?


*   train feature 기간 : 2012 ~ 2018 (최근 7년에 대한 투자지표)
*   train target 기간 : 2019년
*   test feature 기간 : 2013 ~ 2019 (최근 7년에 대한 투자지표)
*   test target 기간 : 2020년


2. 20 개의 feature 대부분 target 간의 상관관계가 매우 떨어지는 것을 확인하였습니다. (statsmodel 회귀 결과 '자산회전율' 1개의 feature 제외) 그리고 이에 따라 정확성 또한 조금 떨어지는 것도 확인하였습니다. 이러한 상황에서 분석의 의의를 찾기위해 어느 방향성으로 분석을 진행해야 하는지요?
